#**Proyecto - Sistemas Recomendadores - IIC3633**

## Implementación en Keras de Session-Based RNNs for Recommendation con soft atenttion

### V2: Implementación de embedding sobre one-hot vectors para entrenamiento más eficiente y modelo más chico

In [1]:
import os
import sys
import subprocess
import math
import pandas as pd
import numpy as np
import sklearn
import psutil
import humanize
import pyreclab
import GPUtil as GPU
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.python.client import device_lib


import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

import warnings
#warnings.filterwarnings("ignore")

import keras
import keras.backend as K
from keras.utils import to_categorical
from keras.losses import cosine_proximity, categorical_crossentropy
from keras.models import Model, Sequential
from keras.initializers import glorot_uniform
from keras.layers.core import Permute, Reshape, RepeatVector
from keras.layers import Input, Dense, Dropout, CuDNNGRU, Embedding, concatenate, Lambda, multiply, merge, Flatten
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [2]:
# Cargamos dataframes preprocesados de MovieLens20MM
PATH_TO_TRAIN = './data/train.csv' #all_
PATH_TO_DEV = './data/dev.csv'
PATH_TO_TEST = './data/test.csv'

train_data = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'ItemId':np.int64})
dev_data = pd.read_csv(PATH_TO_DEV, sep='\t', dtype={'ItemId':np.int64})
test_data = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'ItemId': np.int64})

In [3]:
train_data.head()

,SessionId,ItemId,Time
0,18,186,1267347706
1,18,858,1236356241
2,18,912,1283426281
3,18,1221,1236356224
4,18,1230,1236293194


In [33]:
class SessionDataset:
    def __init__(self, data, sep='\t', session_key='SessionId', item_key='ItemId', time_key='Time', n_samples=-1, itemmap=None, time_sort=True):
        """
        Args:
            n_samples: the number of samples to use. If -1, use the whole dataset.
            itemmap: mapping between item IDs and item indices
            time_sort: whether to sort the sessions by time or not
        """
        self.df = data
        self.session_key = session_key
        self.item_key = item_key
        self.time_key = time_key
        self.time_sort = time_sort
        self.add_item_indices(itemmap=itemmap)
        self.df.sort_values([session_key, time_key], inplace=True)    

        #Sort the df by time, and then by session ID. That is, df is sorted by session ID and
        #clicks within a session are next to each other, where the clicks within a session are time-ordered.

        self.click_offsets = self.get_click_offsets()
        self.session_idx_arr = self.order_session_idx()
        
        
    def get_click_offsets(self):
        """
        Return the offsets of the beginning clicks of each session IDs,
        where the offset is calculated against the first click of the first session ID.
        """
        offsets = np.zeros(self.df[self.session_key].nunique() + 1, dtype=np.int32)
        # group & sort the df by session_key and get the offset values
        offsets[1:] = self.df.groupby(self.session_key).size().cumsum()

        return offsets
    

    def order_session_idx(self):
        """ Order the session indices """
        if self.time_sort:
            # starting time for each sessions, sorted by session IDs
            sessions_start_time = self.df.groupby(self.session_key)[self.time_key].min().values
            # order the session indices by session starting times
            session_idx_arr = np.argsort(sessions_start_time)
        else:
            session_idx_arr = np.arange(self.df[self.session_key].nunique())

        return session_idx_arr
    
    
    def add_item_indices(self, itemmap=None):
        """ 
        Add item index column named "item_idx" to the df
        Args:
            itemmap (pd.DataFrame): mapping between the item Ids and indices
        """
        if itemmap is None:
            item_ids = self.df[self.item_key].unique()  # unique item ids
            item2idx = pd.Series(data=np.arange(len(item_ids)),
                                 index=item_ids)
            itemmap = pd.DataFrame({self.item_key:item_ids,
                                   'item_idx':item2idx[item_ids].values})
        
        self.itemmap = itemmap
        self.df = pd.merge(self.df, self.itemmap, on=self.item_key, how='inner')
        
    
    @property    
    def items(self):
        return self.itemmap.ItemId.unique()
        

class SessionDataLoader:
    def __init__(self, dataset, batch_size=50):
        """
        A class for creating session-parallel mini-batches.
        Args:
             dataset (SessionDataset): the session dataset to generate the batches from
             batch_size (int): size of the batch
        """
        self.dataset = dataset
        self.batch_size = batch_size
        
        
    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,): torch.FloatTensor. Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        # initializations
        df = self.dataset.df
        
        session_key='SessionId'
        item_key='ItemId'
        time_key='TimeStamp'
        self.n_items = df[item_key].nunique()+1
        click_offsets = self.dataset.click_offsets
        session_idx_arr = self.dataset.session_idx_arr

        iters = np.arange(self.batch_size)
        maxiter = iters.max()
        start = click_offsets[session_idx_arr[iters]]
        end = click_offsets[session_idx_arr[iters] + 1]
        mask = [] # indicator for the sessions to be terminated
        finished = False        

        while not finished:
            minlen = (end - start).min()
            # Item indices(for embedding) for clicks where the first sessions start
            idx_target = df.item_idx.values[start]
            for i in range(minlen - 1):
                # Build inputs & targets
                idx_input = idx_target
                idx_target = df.item_idx.values[start + i + 1]
                input = idx_input
                target = idx_target
                yield input, target, mask
                
            # click indices where a particular session meets second-to-last element
            start = start + (minlen - 1)
            # see if how many sessions should terminate
            mask = np.arange(len(iters))[(end - start) <= 1]
            for idx in mask:
                maxiter += 1
                if maxiter >= len(click_offsets) - 1:
                    finished = True
                    break
                # update the next starting/ending point
                iters[idx] = maxiter
                start[idx] = click_offsets[session_idx_arr[maxiter]]
                end[idx] = click_offsets[session_idx_arr[maxiter] + 1]

In [34]:
batch_size = 512 #como en el paper
session_max_len = 100
embeddingp=False

n_items = len(train_data['ItemId'].unique())+1
print("Items unicos training:", n_items)

dev_n_items = len(dev_data['ItemId'].unique())+1
print("Items unicos dev:", dev_n_items)

test_n_items = len(test_data['ItemId'].unique())+1
print("Items unicos testing:", test_n_items)

train_samples_qty = len(train_data['SessionId'].unique()) # cantidad sesiones no augmentadas de train
print("Sesiones training:", train_samples_qty)

dev_samples_qty = len(dev_data['SessionId'].unique()) # cantidad sesiones no augmentadas de dev
print("Sesiones validation:",dev_samples_qty)

test_samples_qty = len(test_data['SessionId'].unique()) # cantidad sesiones no augmentadas de test
print("Sesiones testing:", test_samples_qty)

Items unicos training: 11619
Items unicos dev: 10103
Items unicos testing: 10365
Sesiones training: 19850
Sesiones validation: 5747
Sesiones testing: 5270


In [35]:
train_fraction = 1#256 # 1/fraction es la cantidad de sesiones mas recientes a considerar
dev_fraction = 1#2

train_offset_step=train_samples_qty//batch_size
dev_offset_step=dev_samples_qty//batch_size
test_offset_step=test_samples_qty//batch_size


aux = [0]
aux.extend(list(train_data['ItemId'].unique()))
itemids = np.array(aux)
itemidmap = pd.Series(data=np.arange(n_items), index=itemids) 

In [43]:
# Modelo

# ToDo: self-attention

def attention_3d_block(inputs, TIME_STEPS, SINGLE_ATTENTION_VECTOR=True):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    #a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    return output_attention_mul
    
emb_size = 50
hidden_units = 100
size = emb_size
#size = emb_size if embeddingp else n_items

inputs = Input(batch_shape=(batch_size, 1, n_items))
#emb = Embedding(n_items, emb_size, embeddings_initializer='uniform', input_length=session_max_len)(inputs)
#drop1 = Dropout(0.25)(emb)
gru, gru_states = CuDNNGRU(hidden_units, stateful=True, return_state=True)(inputs)# drop1) #
drop2 = Dropout(0.25)(gru)
#attention_mul = attention_3d_block(drop2, session_max_len)
#attention_mul = Flatten()(attention_mul)
predictions = Dense(n_items, activation='softmax')(drop2)#(attention_mul)#
model = Model(input=inputs, output=[predictions])
#custom_loss = custom_cosine_loss(itemidmap, n_items)
# lr original es 0.0001
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# Try Nadam, too
model.compile(loss=categorical_crossentropy, optimizer=opt)
model.summary()

#filepath='./bast/model_checkpoint'
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=2, save_best_only=True, mode='min')
callbacks_list = []#[checkpoint]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (512, 1, 11619)           0         
_________________________________________________________________
cu_dnngru_5 (CuDNNGRU)       [(512, 100), (512, 100)]  3516300   
_________________________________________________________________
dropout_5 (Dropout)          (512, 100)                0         
_________________________________________________________________
dense_5 (Dense)              (512, 11619)              1173519   
Total params: 4,689,819
Trainable params: 4,689,819
Non-trainable params: 0
_________________________________________________________________


/home/pcerdam/.virtualenvs/KerasRecSysPy3/lib/python3.5/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


In [37]:
def get_states(model):
    return [K.get_value(s) for s,_ in model.state_updates]

def set_states(model, states):
    for (d,_), s in zip(model.state_updates, states):
        K.set_value(d, s)

In [41]:
def get_recall(generator, model, recall_k=20):

    n = 0
    suma = 0
    suma_baseline = 0

    for feat, label, mask in generator:
        input_oh = to_categorical(feat, num_classes=loader.n_items) 
        input_oh = np.expand_dims(input_oh, axis=1)

        target_oh = to_categorical(label, num_classes=loader.n_items)

        pred = model.predict(input_oh, batch_size=batch_size)

        if n%100 == 0:
            try:
                print("{}:{}".format(n, suma/n))
            except:
                pass

        for row_idx in range(feat.shape[0]):
            #baseline_pred = obj.recommend( str(test_batch[0][row_idx][-1]), 20 )
            pred_row = pred[row_idx] # 37484, #.reshape(1, -1) # 50,
            label_row = target_oh[row_idx]        #.reshape(1, -1) # 50,

            idx1 = pred_row.argsort()[-recall_k:][::-1]
            idx2 = label_row.argsort()[-1:][::-1]

            n += 1
            #print(idx1)
            #print(idx2)
            if idx2[0] in idx1:
                suma += 1

            #if idx2[0] in baseline_pred:
            #  suma_baseline += 1

    print("Recall@{} epoch {}: {}".format(recall_k, epoch, suma/n))

In [44]:
train_dataset = SessionDataset(train_data)

for epoch in range(5):
    print("Starting epoch {}...".format(epoch))
    loader = SessionDataLoader(train_dataset, batch_size=batch_size)
    for feat, target, mask in loader:
                
        input_oh = to_categorical(feat, num_classes=loader.n_items) 
        input_oh = np.expand_dims(input_oh, axis=1)
        
        target_oh = to_categorical(target, num_classes=loader.n_items)
        
        tr_loss = model.train_on_batch(input_oh, target_oh)
        
        real_mask = np.ones((batch_size, 1))
        for elt in mask:
            real_mask[elt, :] = 0
        
        hidden_states = get_states(model)[0]#512,100    #get_states(model)[0]
               
        hidden_states = np.multiply(real_mask, hidden_states)
        hidden_states = np.array(hidden_states, dtype=np.float32)
        #hidden_states = np.expand_dims(hidden_states, axis=0)
        
        #set_states(model, hidden_states)\):
        #print(hidden_states.shape)
        #K.set_value(model.layers[1].states, hidden_states)
        model.layers[1].reset_states(hidden_states)
        print("Loss: {}".format(tr_loss))
    
test_dataset = SessionDataset(test_data)
test_generator = SessionDataLoader(test_dataset, batch_size=batch_size)

get_recall(test_generator, model)

Starting epoch 0...
Loss: 9.360455513000488
Loss: 9.35983657836914
Loss: 9.35921859741211
Loss: 9.358386993408203
Loss: 9.357837677001953
Loss: 9.356842994689941
Loss: 9.356256484985352
Loss: 9.35522174835205
Loss: 9.353857040405273
Loss: 9.353036880493164
Loss: 9.352129936218262
Loss: 9.350399017333984
Loss: 9.349419593811035
Loss: 9.34752368927002
Loss: 9.344944953918457
Loss: 9.341999053955078
Loss: 9.3401517868042
Loss: 9.337827682495117
Loss: 9.334299087524414
Loss: 9.332107543945312
Loss: 9.327170372009277
Loss: 9.317972183227539
Loss: 9.312793731689453
Loss: 9.309549331665039
Loss: 9.298288345336914
Loss: 9.287708282470703
Loss: 9.274070739746094
Loss: 9.259845733642578
Loss: 9.229280471801758
Loss: 9.206003189086914
Loss: 9.171366691589355
Loss: 9.129814147949219
Loss: 9.081046104431152
Loss: 8.99659538269043
Loss: 8.881094932556152
Loss: 8.762486457824707
Loss: 8.598175048828125
Loss: 8.365857124328613
Loss: 8.21557331085205
Loss: 8.170770645141602
Loss: 8.197784423828125
Loss

Loss: 7.338674545288086
Loss: 7.5430216789245605
Loss: 7.4605560302734375
Loss: 7.335380554199219
Loss: 7.410472869873047
Loss: 7.422455787658691
Loss: 7.42734432220459
Loss: 7.3845367431640625
Loss: 7.438068389892578
Loss: 7.45200252532959
Loss: 7.409441947937012
Loss: 7.441132068634033
Loss: 7.312017917633057
Loss: 7.311735153198242
Loss: 7.28134822845459
Loss: 7.1991496086120605
Loss: 7.265658378601074
Loss: 7.134681701660156
Loss: 7.052076816558838
Loss: 6.987920761108398
Loss: 7.053654193878174
Loss: 7.032362937927246
Loss: 7.027325630187988
Loss: 7.078124523162842
Loss: 7.1768975257873535
Loss: 7.177218914031982
Loss: 7.199578762054443
Loss: 7.299607276916504
Loss: 7.209402084350586
Loss: 7.185787200927734
Loss: 7.321709632873535
Loss: 7.31127405166626
Loss: 7.338126182556152
Loss: 7.3270158767700195
Loss: 7.396462440490723
Loss: 7.290217876434326
Loss: 7.431671142578125
Loss: 7.407788276672363
Loss: 7.435378074645996
Loss: 7.331201553344727
Loss: 7.410101890563965
Loss: 7.324519

Loss: 7.628222465515137
Loss: 7.546392917633057
Loss: 7.535097122192383
Loss: 7.50856876373291
Loss: 7.559432029724121
Loss: 7.660559177398682
Loss: 7.686315536499023
Loss: 7.5406036376953125
Loss: 7.628374099731445
Loss: 7.460094451904297
Loss: 7.509755611419678
Loss: 7.5309224128723145
Loss: 7.498794078826904
Loss: 7.510526657104492
Loss: 7.483493804931641
Loss: 7.397088050842285
Loss: 7.421879291534424
Loss: 7.477240085601807
Loss: 7.529206275939941
Loss: 7.575039386749268
Loss: 7.632397174835205
Loss: 7.592187881469727
Loss: 7.593867301940918
Loss: 7.605396270751953
Loss: 7.533267021179199
Loss: 7.452160835266113
Loss: 7.415561676025391
Loss: 7.273403167724609
Loss: 7.454428195953369
Loss: 7.539676666259766
Loss: 7.532684803009033
Loss: 7.56278657913208
Loss: 7.501795291900635
Loss: 7.614496231079102
Loss: 7.517552852630615
Loss: 7.551268577575684
Loss: 7.536887168884277
Loss: 7.574742794036865
Loss: 7.5074462890625
Loss: 7.514926433563232
Loss: 7.567038059234619
Loss: 7.5937447547

Loss: 7.298264503479004
Loss: 7.373276710510254
Loss: 7.351160049438477
Loss: 7.328546047210693
Loss: 7.378584384918213
Loss: 7.322612285614014
Loss: 7.326103210449219
Loss: 7.396612644195557
Loss: 7.405399799346924
Loss: 7.365076541900635
Loss: 7.527126312255859
Loss: 7.400423049926758
Loss: 7.4047346115112305
Loss: 7.349693298339844
Loss: 7.349064350128174
Loss: 7.3214826583862305
Loss: 7.401169776916504
Loss: 7.535115718841553
Loss: 7.431544780731201
Loss: 7.339187145233154
Loss: 7.387363910675049
Loss: 7.276678085327148
Loss: 7.379079818725586
Loss: 7.301840305328369
Loss: 7.3553876876831055
Loss: 7.475337982177734
Loss: 7.447378158569336
Loss: 7.492251873016357
Loss: 7.410336971282959
Loss: 7.249166965484619
Loss: 7.413915634155273
Loss: 7.439558982849121
Loss: 7.330807685852051
Loss: 7.435976982116699
Loss: 7.4445061683654785
Loss: 7.393953800201416
Loss: 7.301780700683594
Loss: 7.47413444519043
Loss: 7.350451469421387
Loss: 7.359567642211914
Loss: 7.366846561431885
Loss: 7.37459

Loss: 7.453531742095947
Loss: 7.496488094329834
Loss: 7.37845516204834
Loss: 7.344604015350342
Loss: 7.1783061027526855
Loss: 7.5091938972473145
Loss: 7.385111331939697
Loss: 7.4517822265625
Loss: 7.4404072761535645
Loss: 7.387199401855469
Loss: 7.257209777832031
Loss: 7.350050926208496
Loss: 7.252456188201904
Loss: 7.269032001495361
Loss: 7.343436241149902
Loss: 7.382643222808838
Loss: 7.378782749176025
Loss: 7.386559009552002
Loss: 7.367831230163574
Loss: 7.3881072998046875
Loss: 7.186967849731445
Loss: 7.213690280914307
Loss: 7.269196033477783
Loss: 7.256793022155762
Loss: 7.163329601287842
Loss: 7.359180450439453
Loss: 7.301769256591797
Loss: 7.21165657043457
Loss: 7.304641246795654
Loss: 7.134154319763184
Loss: 7.0536699295043945
Loss: 7.105280876159668
Loss: 7.058825969696045
Loss: 7.048731327056885
Loss: 6.94465446472168
Loss: 7.133984565734863
Loss: 7.093996047973633
Loss: 7.150691032409668
Loss: 7.129056930541992
Loss: 7.117417335510254
Loss: 7.0643310546875
Loss: 7.1071586608

...

Loss: 6.719635963439941
Loss: 6.737818717956543
Loss: 6.891140937805176
Loss: 6.7965407371521
Loss: 6.823192119598389
Loss: 6.801511287689209
Loss: 6.910632133483887
Loss: 6.802165985107422
Loss: 6.799351692199707
Loss: 6.7656450271606445
Loss: 6.832758903503418
Loss: 6.882811069488525
Loss: 6.822427272796631
Loss: 6.820343971252441
Loss: 6.720586776733398
Loss: 6.8388991355896
Loss: 6.559906005859375
Loss: 6.771770000457764
Loss: 6.707880020141602
Loss: 6.705690860748291
Loss: 6.701140403747559
Loss: 6.6237568855285645
Loss: 6.758138179779053
Loss: 6.804620742797852
Loss: 6.729322910308838
Loss: 6.643312454223633
Loss: 6.5992231369018555
Loss: 6.707667350769043
Loss: 6.503514766693115
Loss: 6.677790641784668
Loss: 6.718611717224121
Loss: 6.58797550201416
Loss: 6.598807334899902
Loss: 6.68918514251709
Loss: 6.632897853851318
Loss: 6.640231609344482
Loss: 6.686320781707764
Loss: 6.645367622375488
Loss: 6.863309860229492
Loss: 6.927280426025391
Loss: 6.702864170074463
Loss: 6.73279571533

Loss: 6.713531970977783
Loss: 6.768557548522949
Loss: 6.814712047576904
Loss: 6.819212436676025
Loss: 6.822267532348633
Loss: 6.769907474517822
Loss: 6.741367340087891
Loss: 6.633745193481445
Loss: 6.759777545928955
Loss: 6.608006000518799
Loss: 6.489548683166504
Loss: 6.463000774383545
Loss: 6.739781856536865
Loss: 6.499044418334961
Loss: 6.66870641708374
Loss: 6.591345310211182
Loss: 6.587809085845947
Loss: 6.595247268676758
Loss: 6.6178412437438965
Loss: 6.722270488739014
Loss: 6.721607208251953
Loss: 6.645965099334717
Loss: 6.643126964569092
Loss: 6.84536600112915
Loss: 6.5911054611206055
Loss: 6.684035778045654
Loss: 6.691743850708008
Loss: 6.724052429199219
Loss: 6.835635185241699
Loss: 6.7659687995910645
Loss: 6.6994733810424805
Loss: 6.639719009399414
Loss: 6.731487274169922
Loss: 6.6385040283203125
Loss: 6.629292011260986
Loss: 6.687458515167236
Loss: 6.703699111938477
Loss: 6.633642673492432
Loss: 6.5920090675354
Loss: 6.516701698303223
Loss: 6.751734256744385
Loss: 6.7690777

Loss: 6.774842262268066
Loss: 6.752798557281494
Loss: 6.72030782699585
Loss: 6.6049394607543945
Loss: 6.702709197998047
Loss: 6.775400161743164
Loss: 6.676259994506836
Loss: 6.8149285316467285
Loss: 6.901421546936035
Loss: 6.692112445831299
Loss: 6.769406795501709
Loss: 6.8135552406311035
Loss: 6.827195167541504
Loss: 6.816011428833008
Loss: 6.7669243812561035
Loss: 6.970648765563965
Loss: 6.763710975646973
Loss: 6.9261322021484375
Loss: 6.767989158630371
Loss: 6.776115417480469
Loss: 6.7438249588012695
Loss: 6.6716790199279785
Loss: 6.6586408615112305
Loss: 6.647177696228027
Loss: 6.70228385925293
Loss: 6.774636268615723
Loss: 6.669686317443848
Loss: 6.6486992835998535
Loss: 6.601751804351807
Loss: 6.80908203125
Loss: 6.623026371002197
Loss: 6.674583435058594
Loss: 6.676876544952393
Loss: 6.626522064208984
Loss: 6.670891761779785
Loss: 6.673308849334717
Loss: 6.658700466156006
Loss: 6.688497543334961
Loss: 6.68990421295166
Loss: 6.61729097366333
Loss: 6.6873273849487305
Loss: 6.816219

Loss: 6.828689098358154
Loss: 6.741593360900879
Loss: 6.750237464904785
Loss: 6.665332794189453
Loss: 6.809950828552246
Loss: 6.576658248901367
Loss: 6.671226501464844
Loss: 6.794448375701904
Loss: 6.54340934753418
Loss: 6.785588264465332
Loss: 6.92301607131958
Loss: 6.825022220611572
Loss: 6.808852672576904
Loss: 6.709781646728516
Loss: 6.761818885803223
Loss: 6.838511943817139
Loss: 6.812520503997803
Loss: 6.901211738586426
Loss: 6.849257946014404
Loss: 6.8116865158081055
Loss: 6.781274795532227
Loss: 6.8932576179504395
Loss: 6.929111480712891
Loss: 6.880531311035156
Loss: 6.835445404052734
Loss: 6.815310478210449
Loss: 6.725651741027832
Loss: 6.591629981994629
Loss: 6.681253910064697
Loss: 6.75526762008667
Loss: 6.783989429473877
Loss: 6.796708583831787
Loss: 6.802034854888916
Loss: 6.606452941894531
Loss: 6.6775054931640625
Loss: 6.665562152862549
Loss: 6.697573661804199
Loss: 6.743870735168457
Loss: 6.973447322845459
Loss: 6.699926853179932
Loss: 6.975526332855225
Loss: 6.77703285

Loss: 6.512809753417969
Loss: 6.4761962890625
Loss: 6.370774269104004
Loss: 6.422579765319824
Loss: 6.485744953155518
Loss: 6.342094421386719
Loss: 6.515285491943359
Loss: 6.566721439361572
Loss: 6.45499324798584
Loss: 6.310659885406494
Loss: 6.532066345214844
Loss: 6.647915363311768
Loss: 6.4852447509765625
Loss: 6.337687969207764
Loss: 6.481449604034424
Loss: 6.495675086975098
Loss: 6.632503509521484
Loss: 6.504549026489258
Loss: 6.367791175842285
Loss: 6.4754204750061035
Loss: 6.458425521850586
Loss: 6.502137660980225
Loss: 6.414021015167236
Loss: 6.463896751403809
Loss: 6.375636100769043
Loss: 6.519946098327637
Loss: 6.568024635314941
Loss: 6.618645668029785
Loss: 6.550222396850586
Loss: 6.545757293701172
Loss: 6.573029518127441
Loss: 6.45794677734375
Loss: 6.488950252532959
Loss: 6.4312849044799805
Loss: 6.607262134552002
Loss: 6.394929885864258
Loss: 6.626189231872559
Loss: 6.462006568908691
Loss: 6.730594635009766
Loss: 6.6101603507995605
Loss: 6.631371974945068
Loss: 6.52277040

In [29]:
dev_dataset = SessionDataset(dev_data)
dev_generator = SessionDataLoader(dev_dataset, batch_size=batch_size)
    
get_recall(dev_generator, model)

     SessionId  ItemId        Time  item_idx
0           49    2302  1367549005         0
103         49     163  1367549017         1
208         49    2985  1367549031         2
279         49    1358  1367549044         3
385         49     785  1367549074         4
     SessionId  ItemId        Time  item_idx
0           49    2302  1367549005         0
103         49     163  1367549017         1
208         49    2985  1367549031         2
279         49    1358  1367549044         3
385         49     785  1367549074         4
12800:0.005078125
25600:0.0049609375
38400:0.00453125
51200:0.00478515625
64000:0.004703125
76800:0.0046484375
89600:0.00484375
102400:0.0047265625
115200:0.004895833333333334
128000:0.0049609375
140800:0.0050071022727272724


KeyboardInterrupt: 

# Resultados Movie Lens con Batcher paralelo
Recall@20 epoch 3: 0.252019592604712


# All train set
Recall@10 epoch 29: 0.08087340943113773
Recall@20: 0.10473194236526946

vs Hidasi

Recall @ 20 0.2177499329156604
MRR@20: 0.06513681594077811

Pruebas atencion
Baseline
Recall@20 epoch 49: 0.09473825785928144
MultAttn



# Train Set
Recall@10 epoch ..100?: 0.09546921781437126

Recall@10 epoch 14: 0.06404879908501715

Recall @20 epoch 99: 0.08705440681137724

Con session_max_len = 100:

Recall @20 epoch 9: 0.12195335890718563

Con dwell_time NO FUNCIONA BIEN. Hacer ese supuesto en este dataset no tiene sentido.
